In [ ]:
%matplotlib inline
import pandas
import matplotlib
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Number of future samples to mean for prediction
prediction_window = 24

# Average window_stride elements together to form a single row
window_stride = 12

# Length of the windowed sequence
sequence_length = 24 * 7

# Number of features we take from the data
input_features = 8
num_inputs = input_features

# Number of things we are doing regression to predict
num_outputs = 4

# Input Features
columns = ['hour', 'temp', 'windspd', 'winddir', 'no', 'no2', 'nox', 'o3']

# Read the data
df = pandas.read_csv('d00_single.csv')

# Drop useless columns
df = df.drop(['AQS_Code', 'Latitude', 'Longitude', 'epoch', 'day'], axis=1)

# Unprocessed dataset
nd = df[columns].values

# Windowed dataset
nd_window = np.zeros((int(nd.shape[0] / window_stride), num_inputs))

row = 0
while row < nd.shape[0] - window_stride:
    for i in range(0, input_features):
        nd_window[int(row/window_stride)][i] = np.mean(nd[row:row+window_stride,i])
    row += window_stride
    
scaler = MinMaxScaler()
scaler.fit(nd_window)
nd_window = scaler.transform(nd_window)


# Create sequences
data_features = []
labels = []

rows = deque(maxlen=sequence_length)

for idx, r in enumerate(nd_window):

    rows.append([a for a in r])
    
    # We need the entire sequence filled to make a prediction about the future mean
    if len(rows) < sequence_length:
        continue
    
    # Since we are predicting the mean, make sure we do not go out of bounds in the future
    if idx+1 + prediction_window > nd_window.shape[0]:
        break
        
    data_features.append(rows.copy())
        
    # We are predicting the future mean values
    u_24_no = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 4] )
    u_24_no2 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 5] )
    u_24_nox = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 6] )
    u_24_o3 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 7] )
    
    labels.append([u_24_no, u_24_no2, u_24_nox, u_24_o3])

data_features = np.array(data_features)
labels = np.array(labels)

In [ ]:
from keras.models import Model
from keras import backend as K
from keras.layers import Dense, LSTM, Input, Flatten, Concatenate, Conv2D, Conv1D, MaxPooling2D, Reshape, MaxPooling1D
from keras.callbacks import LearningRateScheduler


import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

layer_input_features = Input(shape=(sequence_length, input_features))

# For some reason putting some extra dimensions before an LSTM works wonders
layer_x1 = Dense(128, input_dim=(sequence_length, input_features))(layer_input_features)
layer_x1 = LSTM(256, return_sequences=True, dropout=0.5)(layer_x1)
layer_x1 = LSTM(256, return_sequences=False, dropout=0.5)(layer_x1)

layer_concat = layer_x1
    
layer_dense = Dense(512, activation='relu')(layer_concat)
layer_output = Dense(num_outputs)(layer_dense)


model = Model(inputs=[layer_input_features], outputs=[layer_output])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[r2])
model.summary()

def sched(epoch, lr):
    new_lr = 0.001 * (0.95 ** epoch)
    print("Epoch(%d) LR: %f" % (epoch+1, new_lr))
    return new_lr

lr_decay = LearningRateScheduler(schedule=sched) 

model.fit(x=data_features, y=labels, batch_size=128, epochs=100, validation_split=0.33, verbose=True, callbacks=[lr_decay])